In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime
import time

In [2]:
def scrape(departureFrom, arrivalTo, departDate, cabinType, requests):
    
    global results
    
    url = "https://www.agoda.com/en-gb/flights/results?cid=1891458&tag=24bd4b3f-b6a1-50d5-e639-d9c49ca41c49&gclid=cj0kcqia7oqrbhd9arisak3uxh0kts4hidqpr1aegsv5v_2puktdlkqbnwfahx2kuewg9gmwkerfz_oaalcyealw_wcb&departureFrom=" + departureFrom + "&departureFromType=1&arrivalTo=" + arrivalTo + "&arrivalToType=1&departDate=" + departDate + "&returnDate=2024-12-31&searchType=1&cabinType=" + cabinType + "&" + "adults=1&sort=8"
    print("\n" + url)
    
    options = webdriver.ChromeOptions()
    agents = ["Chrome/73.0.3683.68","Firefox/66.0.3","Edge/16.16299"]
    print("User agent: " + agents[(requests%len(agents))])
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--user-agent=' + agents[(requests%len(agents))] + '"')  
    
    driver = webdriver.Chrome(options=options)
    driver.implicitly_wait(20)
    driver.get(url)

    # Set the initial scroll height
    scroll_height = 0
    
    # Scroll down until the end (you can adjust the scroll height increment)
    while True:
        # Scroll down by a fixed amount (e.g., 500 pixels)
        driver.execute_script(f"window.scrollTo(0, {scroll_height});")
        time.sleep(5)  # Wait for content to load (adjust as needed)
    
        # Update the scroll height
        scroll_height += 500
    
        # Check if you've reached the bottom (you can adjust this condition)
        if scroll_height >= driver.execute_script("return document.body.scrollHeight"):
            break

    time.sleep(5)   
    soup=BeautifulSoup(driver.page_source, 'lxml')
    
    #get the arrival and departure times
    deptimes = soup.find_all('h3', attrs={'class': 'sc-fzpans sc-fzplWN dazDxU', 'data-testid': 'departure-time'})
    arrtimes = soup.find_all('h3', attrs={'data-testid': 'arrival-time', 'data-testid': ''})
    durations = soup.find_all('span', attrs={'class': 'sc-fzpans sc-fzplWN cyUAcz', 'data-testid': 'duration'})
    
    deptime = []
    for div in deptimes:
        deptime.append(div.getText())    
        
    arrtime = []
    for div in arrtimes:
        arrtime.append(div.getText())   

    duration = []
    for div in durations:
        duration.append(div.getText())  
        
    #Get the price
    price_list = soup.find_all('span', attrs={'class': 'sc-fzpans sc-fzplWN jCUxHt'})
    
    price = []
    for div in price_list:
        price.append(div.getText())

    # Get the airline
    airline_list = soup.find_all('p', attrs={'class': 'sc-fzpans sc-fzplWN blNoJq', 'data-testid': ''})
    
    airline = []
    for div in airline_list:
        airline.append(div.getText())
    
    df = pd.DataFrame({"departureFrom" : departureFrom,
                       "arrivalTo" : arrivalTo,
                       "airline" : [m for m in airline],
                       "startdate" : departDate,
                       "cabintype" : cabinType,
                       "price": [m for m in price],
                       "currency": "VND",
                       "deptime": [m for m in deptime],
                       "arrtime": [m for m in arrtime],
                       "duration": [m for m in duration]
                       })
    results = pd.concat([results, df], sort=False)

    driver.close() #close the browser

    time.sleep(10) #wait 15sec until the next request
    
    return "success"

In [29]:
#Create an empty dataframe 
results = pd.DataFrame(columns=['departureFrom','arrivalTo', 'airline', 'startdate',"cabintype", "price", "currency", 'deptime','arrtime', 'duration'])
scrape('HAN', 'SGN', '2024-05-04', 'Business', 1)
results


https://www.agoda.com/en-gb/flights/results?cid=1891458&tag=24bd4b3f-b6a1-50d5-e639-d9c49ca41c49&gclid=cj0kcqia7oqrbhd9arisak3uxh0kts4hidqpr1aegsv5v_2puktdlkqbnwfahx2kuewg9gmwkerfz_oaalcyealw_wcb&departureFrom=HAN&departureFromType=1&arrivalTo=SGN&arrivalToType=1&departDate=2024-05-04&returnDate=2024-12-31&searchType=1&cabinType=Business&adults=1&sort=8
User agent: Chrome/73.0.3683.68


,departureFrom,arrivalTo,airline,startdate,cabintype,price,currency,deptime,arrtime,duration
0,HAN,SGN,Vietnam Airlines,2024-05-04,Business,"7,786,717",VND,21:00,23:15,2h 15m
1,HAN,SGN,Vietnam Airlines,2024-05-04,Business,"9,004,759",VND,16:00,18:15,2h 15m
2,HAN,SGN,Vietnam Airlines,2024-05-04,Business,"9,004,759",VND,15:00,17:15,2h 15m
3,HAN,SGN,Vietnam Airlines,2024-05-04,Business,"9,004,759",VND,20:00,22:15,2h 15m
4,HAN,SGN,VietJet Air,2024-05-04,Business,"9,033,341",VND,16:30,18:40,2h 10m
5,HAN,SGN,VietJet Air,2024-05-04,Business,"9,033,341",VND,13:20,15:30,2h 10m
6,HAN,SGN,Vietnam Airlines,2024-05-04,Business,"10,006,553",VND,23:00,01:15,2h 15m
7,HAN,SGN,Vietnam Airlines,2024-05-04,Business,"8,031,159",VND,20:05,23:55,3h 50m
8,HAN,SGN,Vietnam Airlines,2024-05-04,Business,"8,252,669",VND,17:00,23:55,6h 55m
9,HAN,SGN,Vietnam Airlines,2024-05-04,Business,"8,363,163",VND,18:55,23:55,5h 0m


In [5]:
# list_airport = ['HAN', 'SGN', 'DAD', 'CXR', 'PQC', 'VDO', 'VII', 'VCA', 'HPH', 'HUI', 'BMV', 'CAH', 'VCL', 'DLI', 'DIN', 'VDH', 'PXU', 'UIH', 'VKG', 'VTG', 'THD']
list_airport = ['HAN', 'SGN', 'DAD', 'NHA', 'VDO', 'VCA']
list_cabin_type = ['Economy', 'Business', 'First', 'Premium economy']

list_date = []
date_1 = date(2024,5,6)
date_2 = date(2024,5,8)
while date_1 <= date_2:
    list_date.append(str(date_1))
    date_1 = date_1 + timedelta(days=1)

results = pd.DataFrame(columns=['departureFrom','arrivalTo', 'airline', 'startdate',"cabintype", "price", "currency", 'deptime','arrtime', 'duration'])
# requests = 0

In [ ]:
for dept_date in list_date:
    for cabin_type in list_cabin_type:
        for dept_airport in list_airport:
            filter_list_airport = list(filter(lambda x: x != dept_airport, list_airport))
            for dest_airport in filter_list_airport:
                requests = 0
                while True:
                    try:
                        scrape(dept_airport, dest_airport, str(dept_date), cabin_type, requests)
                    except ValueError:
                        requests += 1
                        if requests < 3:
                            continue
                        else:
                            break
                    break
results


https://www.agoda.com/en-gb/flights/results?cid=1891458&tag=24bd4b3f-b6a1-50d5-e639-d9c49ca41c49&gclid=cj0kcqia7oqrbhd9arisak3uxh0kts4hidqpr1aegsv5v_2puktdlkqbnwfahx2kuewg9gmwkerfz_oaalcyealw_wcb&departureFrom=HAN&departureFromType=1&arrivalTo=SGN&arrivalToType=1&departDate=2024-05-06&returnDate=2024-12-31&searchType=1&cabinType=Economy&adults=1&sort=8
User agent: Chrome/73.0.3683.68

https://www.agoda.com/en-gb/flights/results?cid=1891458&tag=24bd4b3f-b6a1-50d5-e639-d9c49ca41c49&gclid=cj0kcqia7oqrbhd9arisak3uxh0kts4hidqpr1aegsv5v_2puktdlkqbnwfahx2kuewg9gmwkerfz_oaalcyealw_wcb&departureFrom=HAN&departureFromType=1&arrivalTo=DAD&arrivalToType=1&departDate=2024-05-06&returnDate=2024-12-31&searchType=1&cabinType=Economy&adults=1&sort=8
User agent: Chrome/73.0.3683.68

https://www.agoda.com/en-gb/flights/results?cid=1891458&tag=24bd4b3f-b6a1-50d5-e639-d9c49ca41c49&gclid=cj0kcqia7oqrbhd9arisak3uxh0kts4hidqpr1aegsv5v_2puktdlkqbnwfahx2kuewg9gmwkerfz_oaalcyealw_wcb&departureFrom=HAN&departure

In [7]:
results.to_csv('flight_data2.csv')

In [10]:
results = results.drop_duplicates()
results.to_csv('flight_data.csv')

In [14]:
results.drop_duplicates()

,departureFrom,arrivalTo,airline,startdate,cabintype,price,currency,deptime,arrtime,duration
0,HAN,SGN,VietJet Air,2024-05-05,Economy,"1,536,519",VND,22:50,01:00,2h 10m
1,HAN,SGN,VietJet Air,2024-05-05,Economy,"1,536,519",VND,23:30,01:40,2h 10m
2,HAN,SGN,VietJet Air,2024-05-05,Economy,"1,536,519",VND,05:10,07:20,2h 10m
3,HAN,SGN,VietJet Air,2024-05-05,Economy,"1,536,519",VND,23:05,01:15,2h 10m
4,HAN,SGN,Vietravel Airlines,2024-05-05,Economy,"1,560,337",VND,08:40,10:45,2h 5m
...,...,...,...,...,...,...,...,...,...,...
223,HAN,SGN,Vietnam Airlines,2024-07-11,Economy,"13,632,674",VND,12:05,19:25,7h 20m
224,HAN,SGN,Vietnam Airlines,2024-07-11,Economy,"13,665,576",VND,13:05,17:15,4h 10m
225,HAN,SGN,Vietnam Airlines,2024-07-11,Economy,"13,665,576",VND,12:45,17:15,4h 30m
226,HAN,SGN,Vietnam Airlines,2024-07-11,Economy,"14,837,384",VND,08:55,14:00,5h 5m
